# Case 3: Recheio Customer Data Enrichment & Recommendation Engine

## Overview
This case study focuses on enriching Recheio Cash & Carry’s customer dataset to deliver more relevant product recommendations across digital channels such as the website, mobile app, POS, and contact center. The project combines customer segmentation and association rule mining to develop systems like **Smart Baskets** and **Did You Forget**, enhancing customer experience and increasing Recheio’s share in total customer purchases.

## Business Problem
- Recheio serves two distinct segments (HoReCa and Traditional Retail), each with unique needs.  
- Customer relationship and loyalty are heavily dependent on personalized and insightful interactions.  
- Current data systems are fragmented, limiting the ability to generate impactful recommendations.  
- There is a need to increase **Recheio’s share of wallet** by becoming more relevant in each customer’s purchasing behavior.

---

**This notebook was developed by:**  
- João Venichand - 20211644  
- Gonçalo Custódio - 20211643  
- Diogo Correia - 20211586  
- Duarte Emanuel - 20240564

# 1. Import Libraries

In [85]:
import calendar
import warnings
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import umap
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.patches as patches

# Datasets

In [86]:
clients = pd.read_excel("Case3_Recheio_2025.xlsx",sheet_name=0, header=1)
clients = clients.iloc[:, 1:]
clients

c:\Users\gonca\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
c:\Users\gonca\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


,Client ID,ZIP Code,ID Client Type
0,210100002,4049,NaN
1,210100003,4000,NaN
2,210100004,4149,NaN
3,210100005,4100,Hotelaria
4,210100006,4405,NaN
...,...,...,...
6457,210199921,4750,NaN
6458,210199922,3004,NaN
6459,210199928,4100,NaN
6460,210199937,4520,NaN


In [87]:
products = pd.read_excel("Case3_Recheio_2025.xlsx",sheet_name=1,header=1)
products = products.iloc[:, 1:]
products

c:\Users\gonca\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
c:\Users\gonca\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


,ID Product,Product Description,ID Product Category
0,224780,FARINHA ESPIGA AMIDO MILHO LUS.2KG,FARINHAS
1,276806,LARANJA CAL7 (67/76) RCH,FRUTAS FRESCAS
2,276809,LIMAO CAL 3/4 RCH,FRUTAS FRESCAS
3,277674,COGUMELO BRANCO MÉDIO RCH,LEGUMES FRESCOS
4,277917,TOMATE BB 67/82 1CAM RCH,LEGUMES FRESCOS
...,...,...,...
4578,962871,PETIT GATEAU CHOCOLATE PRETO ROLO 16X80G,BOLOS E SOBREMESAS INDIVIDUAIS
4579,863666,TOSTAS AMANH C/PASSAS 150GR,PRODUTOS DE PADARIA
4580,975316,POLVO FRIBO T7 FLÔR DE 500/800 KG,CEFALÓPODES
4581,617063,NATAS PARMALAT P/CULIN. 1 LT,NATAS


In [88]:
transactions = pd.read_excel("Case3_Recheio_2025.xlsx",sheet_name=2, header=1)
transactions = transactions.iloc[:, 1:]
transactions

c:\Users\gonca\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
c:\Users\gonca\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


,Date,Client ID,ID Product
0,2022-01-02,210100281,224780
1,2022-01-02,210100281,276806
2,2022-01-02,210100281,276809
3,2022-01-02,210100281,277674
4,2022-01-02,210100281,277917
...,...,...,...
884094,2022-12-31,210106386,949447
884095,2022-12-31,210106386,954062
884096,2022-12-31,210199916,106702
884097,2022-12-31,210199916,906800


Check Data Types

In [89]:
clients.dtypes

Client ID          int64
ZIP Code           int64
ID Client Type    object
dtype: object

In [90]:
products.dtypes

ID Product              int64
Product Description    object
ID Product Category    object
dtype: object

In [91]:
transactions.dtypes

Date          datetime64[ns]
Client ID              int64
ID Product             int64
dtype: object

Check Duplicates

In [92]:
duplicates_clients = clients.duplicated().sum()
duplicates_products = products.duplicated().sum()
duplicates_transactions = transactions.duplicated().sum()
print(f"Duplicated rows in Clients: {duplicates_clients}")
print(f"Duplicated rows in Products: {duplicates_products}")
print(f"Duplicated rows in Transactions: {duplicates_transactions}")

Duplicated rows in Clients: 0
Duplicated rows in Products: 0
Duplicated rows in Transactions: 9


Drop Duplicated Rows

In [93]:
transactions = transactions.drop_duplicates()

Check Missing Values

In [94]:
print(clients.isnull().sum())
print(products.isnull().sum())
print(transactions.isnull().sum())

Client ID            0
ZIP Code             0
ID Client Type    5782
dtype: int64
ID Product             0
Product Description    0
ID Product Category    0
dtype: int64
Date          0
Client ID     0
ID Product    0
dtype: int64


# Merge Data

In [95]:
known_clients = clients[clients['ID Client Type'].notnull()]

transactions_known = transactions.merge(
    known_clients[['Client ID', 'ID Client Type']],
    on='Client ID',
    how='inner'
)

data = transactions_known.merge(
    products[['ID Product', 'Product Description']],
    on='ID Product',
    how='left'
)

In [96]:
print(data['ID Client Type'].unique())
data

['Café/Pastelaria' 'Coletiva/Instituição/Cantina' 'Hotelaria'
 'Restaurante - Cozinha Portuguesa' 'Catering' 'Restaurante - Japonês'
 'Bar/Discoteca' 'Restaurante - Churrasqueira' 'Retail Outros'
 'Garrafeira' 'Padaria/Pastelaria' 'Restaurante - Chinês'
 'Restaurante - Cervejaria' 'Restaurante - Steak House' 'Mercearia'
 'Restaurante - Cozinha de Autor' 'Restaurante - Rodízio'
 'Restaurante - Tapas & Petiscos' 'Frutaria' 'Restaurante - Brasileiro'
 'Restaurante - Outra Cozinha Internacional' 'Restaurante - Marisqueira']


,Date,Client ID,ID Product,ID Client Type,Product Description
0,2022-01-02,210100281,224780,Café/Pastelaria,FARINHA ESPIGA AMIDO MILHO LUS.2KG
1,2022-01-02,210100281,276806,Café/Pastelaria,LARANJA CAL7 (67/76) RCH
2,2022-01-02,210100281,276809,Café/Pastelaria,LIMAO CAL 3/4 RCH
3,2022-01-02,210100281,277674,Café/Pastelaria,COGUMELO BRANCO MÉDIO RCH
4,2022-01-02,210100281,277917,Café/Pastelaria,TOMATE BB 67/82 1CAM RCH
...,...,...,...,...,...
389808,2022-12-31,210106386,906800,Hotelaria,MOLHO INGLES UNCLE THOMAS 1 LT
389809,2022-12-31,210106386,912651,Hotelaria,FEIJÃO PRETO UNCLE THOMAS 820 GR
389810,2022-12-31,210106386,915238,Hotelaria,DET LOICA MCHEF 10LT
389811,2022-12-31,210106386,949447,Hotelaria,"BATATA WEDGES C PELE ECOFROST CG 2,5 KG"


## Aqui meter os clusters - por agora fiz manual (HORECA vs Retail)

In [97]:
def map_segment(client_type):
    if pd.isnull(client_type):
        return 'Unknown'
    
    horeca_types = [
        'Café/Pastelaria', 'Coletiva/Instituição/Cantina', 'Hotelaria',
        'Restaurante - Cozinha Portuguesa', 'Catering', 'Restaurante - Japonês',
        'Bar/Discoteca', 'Restaurante - Churrasqueira', 'Restaurante - Chinês',
        'Restaurante - Cervejaria', 'Restaurante - Steak House',
        'Restaurante - Cozinha de Autor', 'Restaurante - Rodízio',
        'Restaurante - Tapas & Petiscos', 'Restaurante - Brasileiro',
        'Restaurante - Outra Cozinha Internacional', 'Restaurante - Marisqueira'
    ]
    
    retail_types = ['Retail Outros', 'Garrafeira', 'Padaria/Pastelaria', 'Mercearia', 'Frutaria']
    
    if client_type in horeca_types:
        return 'HORECA'
    elif client_type in retail_types:
        return 'Retail'
    else:
        return 'Unknown'

data['Client Segment'] = data['ID Client Type'].apply(map_segment)
data

,Date,Client ID,ID Product,ID Client Type,Product Description,Client Segment
0,2022-01-02,210100281,224780,Café/Pastelaria,FARINHA ESPIGA AMIDO MILHO LUS.2KG,HORECA
1,2022-01-02,210100281,276806,Café/Pastelaria,LARANJA CAL7 (67/76) RCH,HORECA
2,2022-01-02,210100281,276809,Café/Pastelaria,LIMAO CAL 3/4 RCH,HORECA
3,2022-01-02,210100281,277674,Café/Pastelaria,COGUMELO BRANCO MÉDIO RCH,HORECA
4,2022-01-02,210100281,277917,Café/Pastelaria,TOMATE BB 67/82 1CAM RCH,HORECA
...,...,...,...,...,...,...
389808,2022-12-31,210106386,906800,Hotelaria,MOLHO INGLES UNCLE THOMAS 1 LT,HORECA
389809,2022-12-31,210106386,912651,Hotelaria,FEIJÃO PRETO UNCLE THOMAS 820 GR,HORECA
389810,2022-12-31,210106386,915238,Hotelaria,DET LOICA MCHEF 10LT,HORECA
389811,2022-12-31,210106386,949447,Hotelaria,"BATATA WEDGES C PELE ECOFROST CG 2,5 KG",HORECA


In [98]:
unique_clients = data['Client ID'].nunique()
unique_clients

429

In [99]:
segment_counts = data['Client Segment'].value_counts()
print(segment_counts)

Client Segment
HORECA    387845
Retail      1968
Name: count, dtype: int64


# Market Basket Analysis (Association Rules)

In [100]:
def run_market_basket_analysis(data, top_n_products=500, min_support=0.01, max_len=2):
    # Step 1: Convert data to basket format (group by order = client + date)
    baskets = data.groupby(['Client ID', 'Date'])['Product Description'].apply(list).reset_index()
    
    # Step 2: Encode transactions to binary matrix
    transactions = baskets['Product Description'].tolist()
    te = TransactionEncoder()
    te_matrix = te.fit(transactions).transform(transactions)
    df_basket = pd.DataFrame(te_matrix, columns=te.columns_)

    # Step 3: Keep only top N most frequent products
    top_products = df_basket.sum().sort_values(ascending=False).head(top_n_products).index
    df_filtered = df_basket[top_products]

    # Step 4: Apply Apriori
    try:
        frequent_itemsets = apriori(
            df_filtered,
            min_support=min_support,
            use_colnames=True,
            max_len=max_len
        )
    except MemoryError:
        print("MemoryError: Try reducing `top_n_products` or `max_len`.")
        return pd.DataFrame()

    # Step 5: Extract association rules
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
    rules = rules[rules['confidence'] > 0.2].sort_values(by='lift', ascending=False)

    return rules

rules_all = run_market_basket_analysis(data, top_n_products=400, max_len=2)
rules_all

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
7441,(IOG PEDAÇOS MORANGO 5KG RTM),(IOG PEDAÇOS PÊSS/MARACUJÁ 5KG RTM),0.014498,0.014046,0.012391,0.854671,60.846490,1.0,0.012187,6.784300,0.998035,0.767081,0.852601,0.868407
7440,(IOG PEDAÇOS PÊSS/MARACUJÁ 5KG RTM),(IOG PEDAÇOS MORANGO 5KG RTM),0.014046,0.014498,0.012391,0.882143,60.846490,1.0,0.012187,8.361836,0.997578,0.767081,0.880409,0.868407
7443,(RISSOIS CAMARAO MINI MCHEF CG 46UN 920G),(PASTEIS BACALHAU MINI MCHEF CG 55UN 825G),0.013896,0.013545,0.011137,0.801444,59.170317,1.0,0.010949,4.968148,0.996953,0.683077,0.798718,0.811833
7442,(PASTEIS BACALHAU MINI MCHEF CG 55UN 825G),(RISSOIS CAMARAO MINI MCHEF CG 46UN 920G),0.013545,0.013896,0.011137,0.822222,59.170317,1.0,0.010949,5.546836,0.996598,0.683077,0.819717,0.811833
7439,(CROQUETES CARNE MINI MCHEF CG 55UN 825G),(PASTEIS BACALHAU MINI MCHEF CG 55UN 825G),0.015351,0.013545,0.011388,0.741830,54.769039,1.0,0.011180,3.820953,0.997047,0.650430,0.738285,0.791285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,(AGUA MINERAL CALDAS DE PENACOVA 5LT),(ACUCAR AMANH BCO PAP KG),0.062005,0.147035,0.012742,0.205502,1.397635,1.0,0.003625,1.073589,0.303313,0.064912,0.068545,0.146081
346,(COUVE CORACAO DE BOI C/FOLHAS RCH),(ACUCAR AMANH BCO PAP KG),0.054028,0.147035,0.011087,0.205200,1.395582,1.0,0.003143,1.073181,0.299642,0.058358,0.068191,0.140300
35,(AGUA MINERAL CALDAS DE PENACOVA 5LT),(LEITE MCHEF UHT M/GORDO LT),0.062005,0.172670,0.013695,0.220874,1.279169,1.0,0.002989,1.061869,0.232669,0.061975,0.058264,0.150094
55,(ARROZ AGULHA MASTERCHEF 5 KG),(LEITE MCHEF UHT M/GORDO LT),0.053125,0.172670,0.011087,0.208687,1.208593,1.0,0.001913,1.045516,0.182275,0.051636,0.043535,0.136447


## Aqui Meter a função de Categorias dos produtos

Graph Visualization Maybe?

# Similarity Measures

Jaccard, Cosine and Dice Similarity

In [101]:
# Step 1: Basket format
baskets = data.groupby(['Client ID', pd.Grouper(key='Date', freq='W')])['Product Description'].apply(list).reset_index()
transactions = baskets['Product Description'].tolist()

# Step 2: Encode
te = TransactionEncoder()
te_matrix = te.fit(transactions).transform(transactions)
df_basket = pd.DataFrame(te_matrix, columns=te.columns_)

# Step 3: Filter by frequency
df_basket = df_basket[df_basket.sum().sort_values(ascending=False).head(1000).index]

In [102]:
def dice_similarity_matrix(X):
    """Custom Dice similarity implementation for binary matrix."""
    X = X.astype(bool).astype(int)
    intersection = np.dot(X.T, X)
    row_sums = X.sum(axis=0).values
    dice = 2 * intersection / (row_sums[:, None] + row_sums[None, :])
    np.fill_diagonal(dice, 1.0)
    return pd.DataFrame(dice, index=X.columns, columns=X.columns)

def compute_similarity_matrices(basket_matrix):
    """
    Compute Jaccard, Cosine, and Dice similarity matrices for binary basket matrix.
    
    Parameters:
        basket_matrix (pd.DataFrame): binary matrix (e.g., clients × products)
    
    Returns:
        dict: {'jaccard': df, 'cosine': df, 'dice': df}
    """
    # Ensure binary
    X = (basket_matrix > 0).astype(int)

    # Jaccard (1 - Jaccard distance)
    jaccard_sim = 1 - pairwise_distances(X.T.values, metric='jaccard')
    df_jaccard = pd.DataFrame(jaccard_sim, index=X.columns, columns=X.columns)

    # Cosine
    cosine_sim = cosine_similarity(X.T.values)
    df_cosine = pd.DataFrame(cosine_sim, index=X.columns, columns=X.columns)

    # Dice
    df_dice = dice_similarity_matrix(X)

    return {
        'jaccard': df_jaccard,
        'cosine': df_cosine,
        'dice': df_dice
    }

In [103]:
similarities = compute_similarity_matrices(df_basket)
print(similarities['jaccard'].head())

c:\Users\gonca\anaconda3\Lib\site-packages\sklearn\metrics\pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


                                  LEITE MCHEF UHT M/GORDO LT  \
LEITE MCHEF UHT M/GORDO LT                          1.000000   
ACUCAR AMANH BCO PAP KG                             0.217148   
FARINHA AMANH S/FERMENTO 1KG                        0.150685   
OVO PASTEURIZADO TETRA BRIK 1 Lt                    0.123157   
MAIONESE GULOSO 5L                                  0.131892   

                                  ACUCAR AMANH BCO PAP KG  \
LEITE MCHEF UHT M/GORDO LT                       0.217148   
ACUCAR AMANH BCO PAP KG                          1.000000   
FARINHA AMANH S/FERMENTO 1KG                     0.233282   
OVO PASTEURIZADO TETRA BRIK 1 Lt                 0.171387   
MAIONESE GULOSO 5L                               0.189581   

                                  FARINHA AMANH S/FERMENTO 1KG  \
LEITE MCHEF UHT M/GORDO LT                            0.150685   
ACUCAR AMANH BCO PAP KG                               0.233282   
FARINHA AMANH S/FERMENTO 1KG                      

In [104]:
print(similarities['cosine'].head())

                                  LEITE MCHEF UHT M/GORDO LT  \
LEITE MCHEF UHT M/GORDO LT                          1.000000   
ACUCAR AMANH BCO PAP KG                             0.357469   
FARINHA AMANH S/FERMENTO 1KG                        0.276446   
OVO PASTEURIZADO TETRA BRIK 1 Lt                    0.232170   
MAIONESE GULOSO 5L                                  0.251589   

                                  ACUCAR AMANH BCO PAP KG  \
LEITE MCHEF UHT M/GORDO LT                       0.357469   
ACUCAR AMANH BCO PAP KG                          1.000000   
FARINHA AMANH S/FERMENTO 1KG                     0.392301   
OVO PASTEURIZADO TETRA BRIK 1 Lt                 0.304193   
MAIONESE GULOSO 5L                               0.336868   

                                  FARINHA AMANH S/FERMENTO 1KG  \
LEITE MCHEF UHT M/GORDO LT                            0.276446   
ACUCAR AMANH BCO PAP KG                               0.392301   
FARINHA AMANH S/FERMENTO 1KG                      

In [105]:
print(similarities['dice'].head())

                                  LEITE MCHEF UHT M/GORDO LT  \
LEITE MCHEF UHT M/GORDO LT                          1.000000   
ACUCAR AMANH BCO PAP KG                             0.356814   
FARINHA AMANH S/FERMENTO 1KG                        0.261905   
OVO PASTEURIZADO TETRA BRIK 1 Lt                    0.219304   
MAIONESE GULOSO 5L                                  0.233047   

                                  ACUCAR AMANH BCO PAP KG  \
LEITE MCHEF UHT M/GORDO LT                       0.356814   
ACUCAR AMANH BCO PAP KG                          1.000000   
FARINHA AMANH S/FERMENTO 1KG                     0.378311   
OVO PASTEURIZADO TETRA BRIK 1 Lt                 0.292622   
MAIONESE GULOSO 5L                               0.318735   

                                  FARINHA AMANH S/FERMENTO 1KG  \
LEITE MCHEF UHT M/GORDO LT                            0.261905   
ACUCAR AMANH BCO PAP KG                               0.378311   
FARINHA AMANH S/FERMENTO 1KG                      

In [106]:
"""plt.figure(figsize=(12, 8))
sns.heatmap(similarities['jaccard'].iloc[:20, :20], cmap="Blues")
plt.title("Jaccard Similarity (Top 20 Products)")
plt.show()"""

'plt.figure(figsize=(12, 8))\nsns.heatmap(similarities[\'jaccard\'].iloc[:20, :20], cmap="Blues")\nplt.title("Jaccard Similarity (Top 20 Products)")\nplt.show()'

# Page Rank

In [107]:
def build_cooccurrence_matrix(df_basket):
    """
    Builds item-item co-occurrence matrix from a binary basket matrix (transactions × products).
    """
    binary = df_basket.astype(int)
    return np.dot(binary.T, binary)

def normalize_transition_matrix(co_matrix):
    """
    Converts a co-occurrence matrix into a row-stochastic transition matrix.
    """
    trans_matrix = co_matrix.astype(float)
    row_sums = trans_matrix.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1
    return trans_matrix / row_sums

def pagerank_scores(trans_matrix, damping=0.85, max_iter=100, tol=1e-6):
    """
    Computes PageRank scores via power iteration.
    """
    n = trans_matrix.shape[0]
    rank = np.ones(n) / n
    teleport = np.ones(n) / n
    for _ in range(max_iter):
        prev_rank = rank.copy()
        rank = damping * trans_matrix.dot(rank) + (1 - damping) * teleport
        if np.linalg.norm(rank - prev_rank, ord=1) < tol:
            break
    return rank

def compute_pagerank_recommendations(df_basket, damping=0.85, top_n=None):
    """
    Full pipeline: from basket matrix to PageRank scores.
    Returns a ranked Series of product scores.
    """
    co_matrix = build_cooccurrence_matrix(df_basket)
    np.fill_diagonal(co_matrix, 0)
    trans_matrix = normalize_transition_matrix(co_matrix)
    scores = pagerank_scores(trans_matrix, damping=damping)
    ranked = pd.Series(scores, index=df_basket.columns).sort_values(ascending=False)
    return ranked if top_n is None else ranked.head(top_n)

In [108]:
pagerank_scores = compute_pagerank_recommendations(df_basket)
pagerank_scores

ESFREGÃO INOX BRAVO 1UN                    0.001
FARINHA AMANH C/FERMENTO 1KG               0.001
MACEDONIA CONG. MCHEF 2,5KG                0.001
NATA UHT CULINARIA 20% MG MCHEF 1LT        0.001
BANANA IMPORTADA CATI CÔR 4 RCH            0.001
                                           ...  
OVO AGRO OVO M 5DUZ IND                    0.001
SALADA RUSSA CONG MCHEF 2,5KG              0.001
LUVAS DESC.NIT.AZ.MCHEF M 100U             0.001
FILETES DE PESCADA DO CABO CG EMB 800GR    0.001
OVOS MCHEF 5DZ M INDUSTRIA                 0.001
Length: 1000, dtype: float64